In [10]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Agentic AI\langenvv\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage


groq_api_key = os.getenv("GROQ_API_KEY")
print("")
model = ChatGroq(model="llama-3.3-70b-versatile", groq_api_key=groq_api_key)

print(model)

response = model.invoke([HumanMessage(content="My name is bharat and i live in pune")])
print(response)




client=<groq.resources.chat.completions.Completions object at 0x00000194D49E5030> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000194D4A4CA90> model_name='llama-3.3-70b-versatile' model_kwargs={} groq_api_key=SecretStr('**********')
content="Namaste Bharat, it's nice to meet you. Pune is a beautiful city, known for its rich cultural heritage and vibrant atmosphere. What do you like to do in your free time, Bharat? Are you a student or working professional?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 47, 'total_tokens': 98, 'completion_time': 0.166499172, 'prompt_time': 0.003483024, 'queue_time': 0.290658205, 'total_time': 0.169982196}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e7667efac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--44aff199-dd9d-43f4-a280-afa7532e789b-0' usage_metadata={'input_tokens': 47, 'output_tokens': 51, 'total_tok

In [12]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="My name is bharat and i live in pune"),
        AIMessage(content="Namaste Bharat, it's nice to meet you. Pune is a wonderful city, known for its rich cultural heritage, educational institutions, and vibrant atmosphere. What do you like to do in your free time, Bharat? Are you a student or working professional?"),
        HumanMessage(content="what is my name?")
        ]
    )

AIMessage(content='Your name is Bharat, and you live in Pune.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 116, 'total_tokens': 129, 'completion_time': 0.025859026, 'prompt_time': 0.007244604, 'queue_time': 0.056290226, 'total_time': 0.03310363}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--cec1d5ce-3474-4c5d-9a38-10decd1c3e29-0', usage_metadata={'input_tokens': 116, 'output_tokens': 13, 'total_tokens': 129})

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model,get_session_history)


In [14]:
config={"configurable":{"session_id":"chat_1"}}


In [15]:

response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Bharat")],
    config=config
    )

In [16]:
response.content

"Hello Bharat! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [17]:
#change the config -- session id to 2
config1 = {"configurable":{"session_id":"chat_2"}}
response = with_message_history.invoke(
    [HumanMessage(content="hello, who am i")],
    config=config1
    )


Prompt Template and chat history

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the quiestions to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages"),
        ]    
    )

chain = prompt | model

In [19]:
chain.invoke({"messages":[HumanMessage(content="My name is bharat")]})

AIMessage(content="Nice to meet you, Bharat! It's a pleasure to assist you. How can I help you today? Do you have any questions or topics you'd like to discuss? I'm all ears!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 59, 'total_tokens': 101, 'completion_time': 0.095964685, 'prompt_time': 0.002885471, 'queue_time': 0.053781328, 'total_time': 0.098850156}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--46018289-b44c-4fa4-ac03-33fcd5de9a06-0', usage_metadata={'input_tokens': 59, 'output_tokens': 42, 'total_tokens': 101})

In [20]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config = {"configurable":{"session_id":"chat_3"}}
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config
    )
response.content

"I don't have that information. I'm a large language model, I don't have the ability to know your personal details, including your name, unless you tell me. If you'd like to share your name with me, I'd be happy to chat with you and use it in our conversation!"

In [22]:
#add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant. Answer all questions to the best of your ability in {language}."
         ),
         MessagesPlaceholder(variable_name="messages"),
         ]
    )
chain = prompt | model

In [23]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is bharat")],"language":"Hindi"})
response.content

'नमस्ते भरत, आपका स्वागत है! मैं आपकी किस प्रकार से मदद कर सकता हूँ?'

Wrap this more complicated chain in a message history class


In [24]:
with_message_history = RunnableWithMessageHistory(
     chain, get_session_history,input_messages_key="messages"
     )

In [25]:
config = {"configurable":{"session_id":"chat_4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am bharat")],"language":"Hindi"},
    config=config
    )
response.content

'नमस्ते भरत, मैं आपकी कैसे मदद कर सकता हूँ?'

In [26]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="whats my name?")],
     "language":"italian"},
     config=config
     )
response.content

'Il tuo nome è Bharat! (Tuo nome è Bharat)'

In [27]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=35,
    strategy="last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
    )

messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="hi! i'am bharat"),
    AIMessage(content="hi! "),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="you are welcome")
    ]
trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='you are welcome', additional_kwargs={}, response_metadata={})]

In [28]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|prompt|model
    )
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what math math problem did i asked?")],
    "language":"English"
    }
    )
response.text()

'You asked for the solution to the math problem "2+2".'

In [29]:
print(prompt.input_variables)

['language', 'messages']


In [30]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness",
        metadata={"source":"mammal-pets-doc"},
        ),
        Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammal-pets-doc"},
        ),
        Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source":"fish-pets-doc"},
        ),
        Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source":"bird-pets-doc"},
        ),
        Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source":"mammal-pets-doc"},
        ),
        ]

In [31]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

vector stores and retirevel

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("groq_api_key") 

llm = ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)

In [33]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [34]:
#vector store
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents, embedding=embeddings)
vectorstore

In [35]:
vectorstore.similarity_search("rabbit")

[Document(id='4b10698d-9144-4944-91a8-487e6db3cec9', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='9bdfaf0d-3525-4ec9-ba9e-07702034dcf3', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='e7bab834-fd26-492a-90f6-bf01942d85e5', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(id='b178fc04-3733-4e8c-b73d-b0cdce4af4de', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [36]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='9bdfaf0d-3525-4ec9-ba9e-07702034dcf3', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e7bab834-fd26-492a-90f6-bf01942d85e5', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

In [37]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
    )
retriever.batch(["cat","dog"])

[[Document(id='9bdfaf0d-3525-4ec9-ba9e-07702034dcf3', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e7bab834-fd26-492a-90f6-bf01942d85e5', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

context:
{context}

"""
prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dogs")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.


In [47]:
print(response)

content='Dogs are great companions, known for their loyalty and friendliness.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 108, 'total_tokens': 123, 'completion_time': 0.026075416, 'prompt_time': 0.0075864, 'queue_time': 0.05523414, 'total_time': 0.033661816}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--b07e760c-e763-4f28-ae3f-5298921fdf37-0' usage_metadata={'input_tokens': 108, 'output_tokens': 15, 'total_tokens': 123}


In [53]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000194F0439FF0>, search_kwargs={'k': 1})